In [10]:
!pip install world_bank_data

In [11]:
# import requests, json

In [12]:
# r=requests.get(
#     'https://api.worldbank.org/v2/country/NG;AG/indicator/FP.CPI.TOTL.ZG?format=json'
# )


In [13]:
# r.json()

In [14]:
import pandas as pd
import world_bank_data as wb
import numpy as np
import json
import requests

In [15]:
df=wb.get_series('FP.CPI.TOTL.ZG')

In [16]:
index=df.index.get_level_values(0).unique()

In [17]:
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.1 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681833 sha256=e8cf86cba4d9fd885df5b886348fb7f55d0c33e5210f0de1040dd2b690dc41d0
  Stored in directory: /Users/finn/Library/Caches/pip/wheels/cd/29/8b/617685ed7942656b36efb06ff9247dbe832e3f4f7724fffc09
Successfully built pycountry


In [18]:
import pycountry

In [19]:
list(pycountry.countries)

[Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533'),
 Country(alpha_2='AF', alpha_3='AFG', flag='🇦🇫', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan'),
 Country(alpha_2='AO', alpha_3='AGO', flag='🇦🇴', name='Angola', numeric='024', official_name='Republic of Angola'),
 Country(alpha_2='AI', alpha_3='AIA', flag='🇦🇮', name='Anguilla', numeric='660'),
 Country(alpha_2='AX', alpha_3='ALA', flag='🇦🇽', name='Åland Islands', numeric='248'),
 Country(alpha_2='AL', alpha_3='ALB', flag='🇦🇱', name='Albania', numeric='008', official_name='Republic of Albania'),
 Country(alpha_2='AD', alpha_3='AND', flag='🇦🇩', name='Andorra', numeric='020', official_name='Principality of Andorra'),
 Country(alpha_2='AE', alpha_3='ARE', flag='🇦🇪', name='United Arab Emirates', numeric='784'),
 Country(alpha_2='AR', alpha_3='ARG', flag='🇦🇷', name='Argentina', numeric='032', official_name='Argentine Republic'),
 Country(alpha_2='AM', alpha_3='ARM', flag='🇦🇲', 

In [20]:
iso3={country.name: country.alpha_3 for country in pycountry.countries}

In [21]:
c3 = {
    'United Kingdom': 'GBR',
    'United States': 'USA',
    'Canada': 'CAN',
    'Australia': 'AUS',
    'China': 'CHN',
    'India': 'IND',
    'Indonesia': 'IDN',
    'Pakistan': 'PAK',
    'Brazil': 'BRA',
    'Nigeria': 'NGA',
    'Bangladesh': 'BGD',
    'Russian Federation': 'RUS',
    'Mexico': 'MEX',
    'Japan': 'JPN',
    'Ethiopia': 'ETH',
    'Philippines': 'PHL',
    'Egypt, Arab Rep.': 'EGY',
    'Vietnam': 'VNM',
    'Congo, Dem. Rep.': 'COD',
    'Turkiye': 'TUR',
    'Iran, Islamic Rep.': 'IRN',
    'Germany': 'DEU',
    'France': 'FRA',
    'Thailand': 'THA'
}

In [22]:
from pathlib import Path

In [23]:
inds = {
    'infl': 'FP.CPI.TOTL.ZG',
    'debt': 'GC.DOD.TOTL.GD.ZS',
    'grow': 'NY.GDP.MKTP.KD.ZG',
    'gdpa': 'NY.GDP.MKTP.KD',
    'gdpp': 'NY.GDP.PCAP.KD',
    'unem': 'SL.UEM.TOTL.ZS',
    'empl': 'SL.EMP.TOTL.SP.ZS',
    # 'empd': 'SL.EMP.TOTL',
    'expo': 'NE.EXP.GNFS.CD',
    'impo': 'NE.IMP.GNFS.CD',
    # 'prod': 'NY.GDP.PCAP.KD.ZG',
    # 'inac': 'SL.UEM.INAK.ZS',
    'popu': 'SP.POP.TOTL',
    # 'wage':''
}
k = 1
i2 = list(inds.keys())[k]
ind = inds[i2]
df = wb.get_series(ind)
index = df.index.get_level_values(0).unique()
iname = df.index.get_level_values(1).unique()[0]
print(i2, ind, iname)


for ind, ind_id in inds.items():
    df = wb.get_series(ind_id)
    iname = df.index.get_level_values(1).unique()[0]
    print(f"{ind} {iname}")


debt GC.DOD.TOTL.GD.ZS Central government debt, total (% of GDP)
infl Inflation, consumer prices (annual %)
debt Central government debt, total (% of GDP)
grow GDP growth (annual %)
gdpa GDP (constant 2015 US$)
gdpp GDP per capita (constant 2015 US$)
unem Unemployment, total (% of total labor force) (modeled ILO estimate)
empl Employment to population ratio, 15+, total (%) (modeled ILO estimate)
expo Exports of goods and services (current US$)
impo Imports of goods and services (current US$)
popu Population, total


In [25]:
def is_numeric(val):
    try:
        float_val = float(val)  # Try to convert the value to float
        # assess for NaN values
        if np.isnan(float_val):
            return False
        return True
    except ValueError:
        return False  # If conversion fails, it is not a numeric value

def parse_countries(df, i2):
    for c in c3:
        i3=c3[c].lower()
        df2=df.loc[c].loc[df.index[0][1]].reset_index().dropna()
        df2.columns=['date', 'value']

        df2['date']=df2['date'].astype(str)+'-01-01'
        df2['value'] = df2['value'].round(3)
        Path("./wb_data/"+i3).mkdir(parents=True, exist_ok=True)
        
        before_len = len(df2)
        df2 = df2[df2["value"].apply(is_numeric)]
        after_len = len(df2)
        if before_len != after_len:
            print(f"dropped {before_len - after_len} rows from {c}")

        
        df2.to_csv("wb_data/" + i3 + '/' + i2 + '.csv', index=False)
        open("wb_data/" + i3+'/'+i2+'.json','w').write(json.dumps(list(df2.T.to_dict().values())))
        print(c)


In [26]:
for i2 in inds:
    ind = inds[i2]
    df = wb.get_series(ind)
    index = df.index.get_level_values(0).unique()
    iname = df.index.get_level_values(1).unique()[0]
    print(i2, ind, iname)
    parse_countries(df, i2)
    print('====================')


infl FP.CPI.TOTL.ZG Inflation, consumer prices (annual %)
United Kingdom
United States
Canada
Australia
China
India
Indonesia
Pakistan
Brazil
Nigeria
Bangladesh
Russian Federation
Mexico
Japan
Ethiopia
Philippines
Egypt, Arab Rep.
Vietnam
Congo, Dem. Rep.
Turkiye
Iran, Islamic Rep.
Germany
France
Thailand
debt GC.DOD.TOTL.GD.ZS Central government debt, total (% of GDP)
United Kingdom
United States
Canada
Australia
China
India
Indonesia
Pakistan
Brazil
Nigeria
Bangladesh
Russian Federation
Mexico
Japan
Ethiopia
Philippines
Egypt, Arab Rep.
Vietnam
Congo, Dem. Rep.
Turkiye
Iran, Islamic Rep.
Germany
France
Thailand
grow NY.GDP.MKTP.KD.ZG GDP growth (annual %)
United Kingdom
United States
Canada
Australia
China
India
Indonesia
Pakistan
Brazil
Nigeria
Bangladesh
Russian Federation
Mexico
Japan
Ethiopia
Philippines
Egypt, Arab Rep.
Vietnam
Congo, Dem. Rep.
Turkiye
Iran, Islamic Rep.
Germany
France
Thailand
gdpa NY.GDP.MKTP.KD GDP (constant 2015 US$)
United Kingdom
United States
Canada
Austr